In [2]:
import clickhouse_connect
import sys
import json
import pandas as pd

CLICKHOUSE_HOSTNAME = '192.168.0.176'
CLICKHOUSE_USER = 'yuri'
CLICKHOUSE_PASSWORD = 'RmonTEiS'

client = clickhouse_connect.get_client(
    host=CLICKHOUSE_HOSTNAME, port=8123, username=CLICKHOUSE_USER, password=CLICKHOUSE_PASSWORD)

print("connected to " + CLICKHOUSE_HOSTNAME + "\n")

QUERY = "select \
A.codprodrelacionado, \
from analytics. _analitica A \
WHERE 1=1 \
and data between toDate(now()) - INTERVAL 90 DAY and toDate(now()) \
--and data toDate('22-09-2023') \
and A.emitente <> '1' \
and A.uf_destinatario IN ('CE') \
and A.cnpjdestinatario <> '07146710000254' \
and A.cnpjdestinatario <> '07608233000110' \
and A.operacao2= 'VENDA'"

with client.query_df_stream(QUERY) as result:
    for row in result:
        print(row)

Unexpected Http Driver Exception


OperationalError: Error HTTPConnectionPool(host='192.168.0.176', port=8123): Max retries exceeded with url: /?wait_end_of_query=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000259CC3EAE50>, 'Connection to 192.168.0.176 timed out. (connect timeout=10)')) executing HTTP request attempt 1 http://192.168.0.176:8123

In [6]:
import cx_Oracle

# cx_Oracle.init_oracle_client(lib_dir="C:\\oracle\\instantclient_21_11")
cx_Oracle.init_oracle_client(lib_dir="C:\\Oracle_Cliente_32Bits")

dsn = cx_Oracle.makedsn('192.168.0.125', '1521', 'ORCL')

conexao = cx_Oracle.connect('LAREDOBI', 'laredo01bi', '192.168.0.125:1521/ORCL')

print("Conectado ao banco de dados em  192.168.0.125")

cursor = conexao.cursor()

consulta = """SELECT \
V.CODPROD AS codprodrelacionado, \
V.CNPJ AS cnpjdestinatario,  \
V.DTSAIDA AS data, \
V.NUMPED AS danfe,\
V.FATURAMENTO AS valor_total, \
V.QT AS quantidade, \
V.PVENDA AS valor_unitario, \
V.Produto AS produtolaredo, \
V.CODFILIAL \
FROM( \
SELECT \
V.CODFILIAL, \
V.FILIAL,  \
V.CODCLI, \
V.RCA, \
V.CODPROD, \
V.DTSAIDA, \
V.CIDADE, \
V.CONDVENDA, \
V."Cod Supervisor", \
V.NUMPED, \
V.NUMREGIAO, \
V.CODCATEGORIA, \
V.QT*V.PVENDA FATURAMENTO, \
V.CODFORNEC, \
V.DATA, \
V.QT, \
V.PESOBRUTO, \
V."Produto" AS Produto, \
V.PVENDA, \
REPLACE(REPLACE(REPLACE(V.CNPJ,'.', ''),'-', ''),'/', '') AS CNPJ, \
CASE WHEN V.CODFILIAL IN(8) THEN V.VLCUSTOREAL * 1.15 ELSE V.VLCUSTOREAL END AS VLCUSTOREAL \
FROM LAREDO.V_VENDASANALITICA_2 V \
WHERE \
V.CONDVENDA = '1' AND \
V."Cod Supervisor" NOT IN  (13, 14, 1, 2, 62, 3, 4)\
AND V.CODFILIAL IN('1','2','22','3','4','5','6','7','8') \
AND V.DTSAIDA = TO_DATE('25/09/2023', 'DD/MM/YYYY')\
 ) V"""

cursor.execute(consulta)

resultados = cursor.fetchall()

dfo = pd.DataFrame(resultados)

cursor.close()
conexao.close()

dfo.head(30)

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "C:\app\client\Administrador\product\19.0.0\client_1\bin\oci.dll is not the correct architecture". See https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html for help